# 21M.387 Fundamentals of Music Processing
## Problem Set 1: Music Representation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
from ipywidgets import interact
import sys
sys.path.append("..")
import fmplib as fmp

plt.rcParams['figure.figsize'] = (12, 4)
fmp.documentation_button()

## Exercise 1

Below is a video of Lang Lang, world famous concert pianist, playing Frederic Chopin's "Minute Waltz".

The sheet music is [here](data/chopin_waltz_op64_1.pdf).

Questions:
1. How long is Lang Lang's rendition of this piece? (approximately, in seconds)
1. How many measures are there in the score? (repeats are not double counted)
1. How many measures are actually played in this recording? (repeated bars are counted here)
1. What is the average tempo of this recording? Give your answer in BPM where a beat is an entire measure. 
1. Based on this average tempo, how long (in milliseconds) does an eighth-note last?
1. In the actual recording, Lang Lang's tempo fluctuates a lot. What is (approximately) his maximum tempo and his minimum tempo?


In [ ]:
ipd.YouTubeVideo('hKILwVH_MdM')

Answers:


## Exercise 2

Using the same score as in Exercise 1, write out what the Standard MIDI File (SMF) representation would look like for measure 5 of the Chopin Waltz (this is the bar where the left hand first comes in). 

- Assume a "ticks per quarter" value of 120.
- Remember that each midi event's tick value is a delta tick from the previous event.
- Assign channel 1 to the left hand and channel 2 to the right hand.
- Use note velocity = 60 for the beginning, but note that there is a slight crescendo at the end of the bar.
- Create your answer as a python array of events, where each event is a tuple of the format:
    `(<delta-tick>, <event-type>, <channel>, <pitch>, <velocity>)`
- It should look a bit like [_Fig 1.13b_](data/midi_table.png) of the text.

The first event is created for you as an example.

In [ ]:
chopin_sequence = [
    (0, 'note_on', 2, 67, 60),
]

## Exercise 3

1. Write the function `pitch_to_freq` that returns the frequency (in Hertz) of a given MIDI pitch, assuming equal tempered tuning. Test it with `p = 69` (answer should be 440.0) and a few other values.

1. Write the function `freq_to_pitch` that returns the midi-pitch from any frequency (also with equal tempered tuning). The returned midi-pitch value should be a floating-point value. Do not round it to the nearest integer.

In [ ]:
def pitch_to_freq(p):
    return 0

# write some testing code here...

def freq_to_pitch(f):
    return 0

# write some testing code here...

## Exercise 4

1. Write the function `pitch_to_spn()` that takes as input an integer midi pitch and returns its _Scientific Pitch Notation_ name as a string. For example, `69` should return the string `"A4"`. In the case of enharmonics, you can choose either the sharp or flat version. For example you can return `"Eb4"` or `"D#4"`.

1. Write the function `g_major_scale()` that returns a list of notes of the G
major scale, starting at G3 and ending at G4. Each note should be the tuple `(<SPN-name>, <note-frequency>)` 

In [ ]:
def pitch_to_spn(s) :
    pass

def g_major_scale() :
    return []

for n in g_major_scale():
    print(n)

## Exercise 5

Write the function `calc_harmonic_intontations` that returns a list of the first N harmonics of a given midi pitch. For each harmonic, it should calculate the tuple `(<f>, <nen>, <delta>)` where:
- `<f>` is the harmonic's frequency
- `<nen>` is the nearest equal-tempered pitch for that frequency, represented in SPN
- `<delta>` is the difference in from `<nen>`'s pitch to `<f>`'s pitch, measured in cents (there are 100 cents in one semitone).

By looking at these data, you can observe when an equal-tempered note might sound a bit out of tune (slightly sharp or flat) when played against another note that has strong harmonic overtones.

In [ ]:
def calc_harmonic_intontations(pitch, num_h) :
    return []

for h in calc_harmonic_intontations(48, 15):
    print(h)

## Exercise 6

How much space, measured in bytes, does it take to store a WAVE file (just the data part, ignoring the WAVE header) for the following cases:

1. A one minute CD quality song (stereo, 16 bits per channel, $F_s = 44100$)
1. 20 seconds of a low quality voice recording (mono, 8 bits per channel, $F_s = 8000$)

Show your actual calculations in python below.

In [ ]:
# you can type the python code which prints the answers


## Exercise 7

Since the decibel scale is logarithmic, it theoretically cannot represent absolute silence (ie, amplitude 0), since $A = 20 \log_{10}(0)$ is not a number ($- \infty$). However, in practice, with audio signals encoded as digital numbers with finite precision, it is possible to guarantee $A=0$ using attenuation in decibels. 

If an audio signal is represented using CD-quality audio (16 bits of precision), what is the smallest amount of signal attenuation, in dB, that guarantees actual silence, where the signal's attenuated values all become $0$? Explain your reasoning.

What attenuation value, in dB, would be needed for 24-bit precision numbers?

Answer:


## Exercise 8

Write the function `make_tone` that synthesizes a tone using additive synthesis, sampled at 44,100 Hz.

The inputs are function should be:
- `f0`: the fundamental frequency (in Hz)
- `dur`: the duration of the produced tone, in seconds
- `partials`: a list of partials where each partial is a tuple of two numbers `(p, a)`:
  - `p`: a partial number (ie, 1 for the fundamental, 2 for the 2nd harmonic, etc...)
  - `a`: an amplitude for that partial ($0 < a \le 1$)

Your code should create the appropriate sinusoid wave for each partial in the list, add them all together, and return the result as a `numpy` array that you can play with `ipd.Audio()`.

In [ ]:
def make_tone(f0, dur, partials):
    return np.zeros(10000)

For each of `partials0`, `partials1`, and `partials2` below:  

- Synthesize a 3 second tone
- Listen to the tone
- Plot the first 1000 samples of the tone
- Describe briefly what each one sounds like. 

Why does `partials2` sound different from the first two? It's also fun to run the code for `partials2` a bunch of times to hear the different versions caused by the random variable.

In [ ]:
partials0 =  [(n, 1./n) for n in range(1,20)]
partials1 =  [(n*2-1, 1./(n*2-1) ) for n in range(1,10)]
partials2 =  [(n + np.random.random(), 1./(n)) for n in range(1,20)]

In [ ]:
# Audio, Plots, Descriptions:


## Exercise 9
This exercise is worth twice the points as the others.

Write the function `synthesize_midi` that synthesizes a MIDI sequence. 

The input are: 
- `sequence`: a list of MIDI events, exactly as specified in Exercise 2. 
- `partials`: a list of partials, describing the desired timbre, in the same format used by Exercise 8.

Each synthesized note should use `make_tone` above with the proper duration, multiplied by a decay envelope, similarly to what you did in lab. Remember to apply the note velocity to each note. You will also need to pick a reasonable tempo.

To test your function, use the `chopin_sequence` that you created in Exercise 2. 

The output should be a `numpy` array that you can play with `ipd.Audio()`.


In [ ]:
def synthesize_midi(sequence, partials) :
    pass


In [ ]:
# test it:

x = synthesize_midi(chopin_sequence, partials1)
print(x.shape)
ipd.Audio(x, rate = 44100)